Async IO with docker
-> faas in python
https://pypi.python.org/pypi/aiodocker/0.8.2
http://aiodocker.readthedocs.io/en/latest/
https://curio.readthedocs.io/en/latest/tutorial.html

In [ ]:
import asyncio
from aiodocker.docker import Docker
from aiodocker.exceptions import DockerError


async def demo(docker):
    try:
        await docker.images.get('alpine:latest')
    except DockerError as e:
        if e.status == 404:
            await docker.pull('alpine:latest')
        else:
            print('Error retrieving alpine:latest image.')
            return

    config = {
        # "Cmd": ["/bin/ash", "-c", "sleep 1; echo a; sleep 1; echo a; sleep 1; echo a; sleep 1; echo x"],
        "Cmd": ["/bin/ash"],
        "Image": "alpine:latest",
        "AttachStdin": True,
        "AttachStdout": True,
        "AttachStderr": True,
        "Tty": False,
        "OpenStdin": True,
        "StdinOnce": True,
    }
    container = await docker.containers.create_or_replace(
        config=config, name='aiodocker-example')
    print("created and started container {}".format(container._id[:12]))

    try:
        ws = await container.websocket(stdin=True, stdout=True, stderr=True, stream=True)
        await container.start()

        async def _send():
            await asyncio.sleep(0.5)
            await ws.send_bytes(b'echo "hello world"\n')
            print("sent a shell command")

        asyncio.ensure_future(_send())
        resp = await ws.receive()
        print("received: {}".format(resp))
        await ws.close()

        output = await container.log(stdout=True)
        print("log output: {}".format(output))
    finally:
        print("removing container")
        await container.delete(force=True)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
from aiodocker.docker import Docker


async def demo(docker):
    print('--------------------------------')
    print('- Check Docker Version Information')
    data_version = await docker.version()
    for key, value in data_version.items():
        print(key, ':', value)

    print('--------------------------------')
    print('- Check Docker Image List')
    images = await docker.images.list()
    for image in images:
        for key, value in image.items():
            if key == 'RepoTags':
                print(key, ':', value)

    print('--------------------------------')
    print('- Check Docker Container List')
    containers = await docker.containers.list()
    for container in containers:
        container_show = await container.show()
        for key, value in container_show.items():
            if key == 'Id':
                print('Id', ':', value[:12])
    print('--------------------------------')


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
from aiodocker.docker import Docker
from aiodocker.exceptions import DockerError


async def demo(docker):
    try:
        await docker.images.get('alpine:latest')
    except DockerError as e:
        if e.status == 404:
            await docker.pull('alpine:latest')
        else:
            print('Error retrieving alpine:latest image.')
            return

    subscriber = docker.events.subscribe()

    config = {
        "Cmd": ["tail", "-f", "/var/log/dmesg"],
        "Image":"alpine:latest",
         "AttachStdin": False,
         "AttachStdout": True,
         "AttachStderr": True,
         "Tty": False,
         "OpenStdin": False,
         "StdinOnce": False,
    }
    container = await docker.containers.create_or_replace(
        config=config, name='testing')
    await container.start()
    print("=> created and started container {}".format(container._id[:12]))

    while True:
        event = await subscriber.get()
        if event is None:
            break

        for key, value in event.items():
            print(key,':', value)

        # Demonstrate simple event-driven container mgmt.
        if event['Actor']['ID'] == container._id:
            if event['Action'] == 'start':
                await container.stop()
                print("=> killed {}".format(container._id[:12]))
            elif event['Action'] == 'stop':
                await container.delete(force=True)
                print("=> deleted {}".format(container._id[:12]))
            elif event['Action'] == 'destroy':
                print('=> done with this container!')
                break

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        # do our stuffs.
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
import aiodocker

async def list_things():
    docker = aiodocker.Docker()
    print('== Images ==')
    for image in (await docker.images.list()):
        tags = image['RepoTags'][0] if image['RepoTags'] else ''
        print(image['Id'], tags)
    print('== Containers ==')
    for container in (await docker.containers.list()):
        print(f" {container._id}")
    await docker.close()

async def run_container():
    docker = aiodocker.Docker()
    print('== Running a hello-world container ==')
    container = await docker.containers.create_or_replace(
        config={
            'Cmd': ['/bin/ash', '-c', 'echo "hello world"'],
            'Image': 'alpine:latest',
        },
        name='testing',
    )
    await container.start()
    logs = await container.log(stdout=True)
    print(''.join(logs))
    await container.delete(force=True)
    await docker.close()

if __name__ == '__main__':
    loop2 = asyncio.get_event_loop()
    loop2.run_until_complete(list_things())
    loop2.run_until_complete(run_container())
    loop2.close()